# SQLDatabaseChain Tutorial by Bhavesh🦜
[**Link to my YouTube Channel**](https://www.youtube.com/BhaveshBhatt8791?sub_confirmation=1)

redshift+redshift_connector://awsuser:***@<clusterid>.xxxxxx.<aws-region>.redshift.amazonaws.com:5439/dev

## Installs

In [ ]:
%%capture
!pip install -q langchain
!pip install -q openai

# Secret Key

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-'

# Converting Pandas to SQL Db

In [ ]:
# import numpy as np
# import pandas as pd
# import sqlite3

# df = pd.read_csv("data.csv")

# df.rename(columns={'User ID': 'user_id',
#                    'Product ID': 'product_id',
#                    'Product Name' : 'product_name',
#                    'Brand' : 'brand',
#                    'Category' : 'category',
#                    'Price' : 'price',
#                    'Color' : 'color',
#                    'Size' : 'size'}, inplace=True)

# conn = sqlite3.connect('fashion_db.sqlite')
# c = conn.cursor()

# c.execute('CREATE TABLE IF NOT EXISTS fashion_products (user_id int, product_id int, product_name text, brand text, category text, price int, rating float, color text, size text)')
# conn.commit()

# df.to_sql('fashion_products', conn, if_exists='replace', index = False)

# c.execute('''
# SELECT product_name FROM fashion_products LIMIT 100
#           ''')

# for row in c.fetchall():
#     print (row)

# Reading the DB

In [ ]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

In [ ]:
import sqlite3
read_sql_query('SELECT * FROM fashion_products LIMIT 10;',
               "fashion_db.sqlite")

(19, 1, 'Dress', 'Adidas', "Men's Fashion", 40, 1.0431592108361825, 'Black', 'XL')
(97, 2, 'Shoes', 'H&M', "Women's Fashion", 82, 4.026416271141911, 'Black', 'L')
(25, 3, 'Dress', 'Adidas', "Women's Fashion", 44, 3.337937559377053, 'Yellow', 'XL')
(57, 4, 'Shoes', 'Zara', "Men's Fashion", 23, 1.0495229563128543, 'White', 'S')
(79, 5, 'T-shirt', 'Adidas', "Men's Fashion", 79, 4.302773408398684, 'Black', 'M')
(98, 6, 'Dress', 'Adidas', "Men's Fashion", 47, 1.3795657395330458, 'Yellow', 'L')
(16, 7, 'Jeans', 'Gucci', "Men's Fashion", 37, 1.3567503746842564, 'White', 'XL')
(63, 8, 'Sweater', 'Zara', "Kids' Fashion", 64, 4.36030328941572, 'Blue', 'XL')
(96, 9, 'Sweater', 'H&M', "Men's Fashion", 53, 4.466181876278437, 'Green', 'XL')
(36, 10, 'T-shirt', 'Zara', "Kids' Fashion", 55, 4.093234402033421, 'White', 'XL')


# Langchain SQL Agent

In [ ]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [ ]:
input_db = SQLDatabase.from_uri('sqlite:///fashion_db.sqlite')
llm_1 = OpenAI(temperature=0)

In [ ]:
db_agent = SQLDatabaseChain(llm = llm_1,
                            database = input_db,
                            verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain/chains/sql_database/base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


# Question 1

In [ ]:
db_agent.run("how many rows are there?")



> Entering new  chain...
how many rows are there?
SQLQuery:SELECT COUNT(*) FROM fashion_products;
SQLResult: [(1000,)]
Answer:There are 1000 rows in the fashion_products table.
> Finished chain.


'There are 1000 rows in the fashion_products table.'

# Question 2

In [ ]:
db_agent.run("how many entries of Adidas are present?")



> Entering new  chain...
how many entries of Adidas are present?
SQLQuery:SELECT COUNT(*) FROM fashion_products WHERE brand = 'Adidas';
SQLResult: [(198,)]
Answer:There are 198 entries of Adidas.
> Finished chain.


'There are 198 entries of Adidas.'

# Question 3

In [ ]:
db_agent.run("how many XL products of Nike are there that have a rating of more than 4?")



> Entering new  chain...
how many XL products of Nike are there that have a rating of more than 4?
SQLQuery:SELECT COUNT(*) FROM fashion_products WHERE brand = 'Nike' AND size = 'XL' AND "Rating" > 4 LIMIT 5;
SQLResult: [(12,)]
Answer:There are 12 XL products of Nike that have a rating of more than 4.
> Finished chain.


'There are 12 XL products of Nike that have a rating of more than 4.'

# Question 4

In [ ]:
db_agent.run("Give all the details of Adidas which have a size of L and have a rating of more than 4.2")



> Entering new  chain...
Give all the details of Adidas which have a size of L and have a rating of more than 4.2
SQLQuery:SELECT * FROM fashion_products WHERE brand = 'Adidas' AND size = 'L' AND "Rating" > 4.2;
SQLResult: [(100, 166, 'Sweater', 'Adidas', "Kids' Fashion", 96, 4.743823985146882, 'White', 'L'), (95, 327, 'Dress', 'Adidas', "Kids' Fashion", 72, 4.572943412760792, 'White', 'L'), (20, 808, 'T-shirt', 'Adidas', "Women's Fashion", 26, 4.324805158780109, 'Yellow', 'L'), (37, 928, 'Sweater', 'Adidas', "Kids' Fashion", 10, 4.585559724465988, 'Black', 'L')]
Answer:Adidas products with size L and rating more than 4.2 are Sweater, Dress, T-shirt and Sweater.
> Finished chain.


'Adidas products with size L and rating more than 4.2 are Sweater, Dress, T-shirt and Sweater.'